In [17]:
import sys

import pandas as pd

from pathlib import Path
from from_root import from_root

sys.path.insert(0, str(from_root("src")))

from utils import list_xlsx_files
from read_and_write_docs import read_excel_sheets

In [18]:
models = ["gpt2"]

corpora = ["Wiki", "Enron", "Perverted Justice", "StackExchange", "ACL", "TripAdvisor", "The Apricity", "Koppel's Blogs"]

data_types = ["training", "test"]

base_loc = "/Volumes/BCross/av_datasets_experiments/ngram_masking_logrpobs"

metadata_base_loc = "/Volumes/BCross/datasets/author_verification"

save_file_name = "raw_all_tokens.xlsx"

In [19]:
base_loc = Path(base_loc)

tokens = []

for data_type in data_types:
    for corpus in corpora:
        for model in models:
            print(f"Wokring on {data_type} - {corpus} - {model}")

            file_dir = base_loc / data_type / corpus / model / "raw_100"
            if not file_dir.exists():
                continue

            excel_files = list_xlsx_files(file_dir)
            for f in excel_files:
                file_loc = file_dir / f

                try:
                    data = read_excel_sheets(file_loc, sheet_names=["phrase score"])
                    phrase_scores = data["phrase score"]
                except Exception:
                    continue

                # keep only what you want
                distinct_phrases = phrase_scores[["phrase", "tokens", "num_tokens"]]
                tokens.append(distinct_phrases)

# concat once, then drop duplicates once at the end
if tokens:
    full_tokens = pd.concat(tokens, ignore_index=True).drop_duplicates().reset_index(drop=True)
else:
    full_tokens = pd.DataFrame(columns=["phrase", "tokens", "num_tokens"])


Wokring on training - Wiki - gpt2
Wokring on training - Enron - gpt2
Wokring on training - Perverted Justice - gpt2
Wokring on training - StackExchange - gpt2
Wokring on training - ACL - gpt2
Wokring on training - TripAdvisor - gpt2
Wokring on training - The Apricity - gpt2
Wokring on training - Koppel's Blogs - gpt2
Wokring on test - Wiki - gpt2
Wokring on test - Enron - gpt2
Wokring on test - Perverted Justice - gpt2
Wokring on test - StackExchange - gpt2
Wokring on test - ACL - gpt2
Wokring on test - TripAdvisor - gpt2
Wokring on test - The Apricity - gpt2
Wokring on test - Koppel's Blogs - gpt2


In [24]:
full_tokens = full_tokens.copy()

full_tokens["phrase_len"] = full_tokens["phrase"].astype(str).str.len()

full_tokens = (
    full_tokens
    .sort_values(["num_tokens", "phrase_len"], ascending=[True, True])
    .drop(columns=["phrase_len"])
    .reset_index(drop=True)
)


In [26]:
save_loc = base_loc / save_file_name
full_tokens.to_excel(save_loc, index=False, engine="openpyxl")